# yacht_reg_keras.py
# regression on the Yacht Hull dataset
# Keras 2.1.5 over TensorFlow 1.7.0, Python 3.5.2
# to use CNTK issue >SET KERAS_BACKEND=cntk (broken)

In [1]:
import numpy as np
import keras as K
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'  # suppress CPU notification

# ------------------------------------------------------------

Using TensorFlow backend.


In [2]:
class MyLogger(K.callbacks.Callback):
  def __init__(self, n, data_x, data_y, pct_close):
    self.n = n   # print loss & acc every n epochs
    self.data_x = data_x
    self.data_y = data_y
    self.pct_close = pct_close

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.n == 0:
      curr_loss = logs.get('loss')  # loss on curr batch, not entire dataset
      total_acc = my_accuracy(self.model, self.data_x, self.data_y, self.pct_close)
      print("epoch = %4d  curr batch loss (mse) = %0.6f  overall acc = %0.2f%%" % (epoch, curr_loss, total_acc * 100))

def my_accuracy(model, data_x, data_y, pct_close):
  num_correct = 0; num_wrong = 0
  n = len(data_x)
  for i in range(n):
    predicted = model.predict(np.array([data_x[i]], dtype=np.float32) )  # [[ x ]]
    actual = data_y[i]
    if np.abs(predicted[0][0] - actual) < np.abs(pct_close * actual):
      num_correct += 1
    else:
      num_wrong += 1
  return (num_correct * 1.0) / (num_correct + num_wrong)

# ------------------------------------------------------------

# 0. get started

In [3]:
print("\nYacht Hull regression example using Keras \n")
np.random.seed(1)

kv = K.__version__
print("Using Keras: ", kv, "\n")


Yacht Hull regression example using Keras 

Using Keras:  2.1.5 



# 1. load data

In [4]:
# |predictors	 0.560000   0.228571 . . 0.692308  |resistance  0.117930
#   [0]          [1]       [2]          [6]         [7]         [8]
train_file = ".\\Data\\yacht_train_mm_tab_cntk.txt"  # 277 items
test_file = ".\\Data\\yacht_test_mm_tab_cntk.txt"  # 31 items

train_x = np.loadtxt(train_file, delimiter="\t", usecols=[1,2,3,4,5,6])
train_y = np.loadtxt(train_file, delimiter="\t", usecols=[8])
test_x = np.loadtxt(test_file, delimiter="\t", usecols=[1,2,3,4,5,6])
test_y = np.loadtxt(test_file, delimiter="\t", usecols=[8])

In [14]:
train_x[:10] #explore first 10 values in the training dataset

array([[0.52  , 0.5429, 0.    , 0.0669, 0.4615, 0.2308],
       [0.54  , 1.    , 0.    , 0.5591, 0.    , 0.7692],
       [1.    , 1.    , 0.55  , 0.563 , 0.4615, 0.3077],
       [0.    , 1.    , 0.55  , 0.563 , 0.4615, 0.    ],
       [1.    , 0.    , 0.55  , 0.3701, 0.4615, 0.5385],
       [1.    , 0.5   , 0.95  , 0.4449, 0.8571, 0.7692],
       [0.54  , 0.    , 0.525 , 0.3425, 0.4725, 0.0769],
       [1.    , 1.    , 0.55  , 0.563 , 0.4615, 0.2308],
       [0.    , 0.    , 0.55  , 0.3701, 0.4615, 0.6154],
       [0.52  , 0.6286, 0.025 , 0.4528, 0.033 , 0.1538]])

# 2. define 6-(10-10)-1 deep NN model

In [6]:
my_init = K.initializers.glorot_uniform(seed=1)
model = K.models.Sequential()
# implicit input layer is here
model.add(K.layers.Dense(units=10, input_dim=6, activation='tanh', kernel_initializer=my_init))  # hidden layer
model.add(K.layers.Dense(units=10, activation='tanh', kernel_initializer=my_init))  # hidden layer
model.add(K.layers.Dense(units=1, activation=None, kernel_initializer=my_init))  # output layer, 1 node for regression

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                70        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


# 3. compile model

In [8]:
basic_sgd = K.optimizers.SGD()
model.compile(loss='mean_squared_error', optimizer=basic_sgd, metrics=['mse'])

# 4. train model

In [9]:
max_epochs = 1000  # touch all test items one time
my_logger = MyLogger(int(max_epochs/10), train_x, train_y, 0.15)  # evaluate accuracy every n epochs
h = model.fit(train_x, train_y, batch_size=16, epochs=max_epochs, verbose=0, callbacks=[my_logger])

epoch =    0  curr batch loss (mse) = 0.171641  overall acc = 5.78%
epoch =  100  curr batch loss (mse) = 0.019916  overall acc = 7.58%
epoch =  200  curr batch loss (mse) = 0.018282  overall acc = 7.58%
epoch =  300  curr batch loss (mse) = 0.016351  overall acc = 9.03%
epoch =  400  curr batch loss (mse) = 0.014161  overall acc = 9.75%
epoch =  500  curr batch loss (mse) = 0.011589  overall acc = 9.75%
epoch =  600  curr batch loss (mse) = 0.008320  overall acc = 13.00%
epoch =  700  curr batch loss (mse) = 0.005595  overall acc = 20.22%
epoch =  800  curr batch loss (mse) = 0.003872  overall acc = 20.94%
epoch =  900  curr batch loss (mse) = 0.002988  overall acc = 20.22%


# 5. evaluate model

In [10]:
acc = my_accuracy(model, test_x, test_y, 0.15)  # within x pct of actual
print("\nFinal overall accuracy on test data  = %0.2f%% " % (acc * 100))
eval_results = model.evaluate(test_x, test_y, verbose=0)  # [0] = loss (mse), [1] = compile-metrics = 'mse' again
print("Final overall loss (mse) on test data = %0.6f" % eval_results[0])


Final overall accuracy on test data  = 19.35% 
Final overall loss (mse) on test data = 0.002645


# 6. save model

In [11]:
mp = ".\\Models\\yacht_model.h5"
model.save(mp)

# 7. make a prediction for previously unseen data

In [12]:
inpts = np.array([[0.5611, 0.2285, 0.9511, 0.0011, 0.4615, 0.9999]], dtype=np.float32)
pred = model.predict(inpts)
np.set_printoptions(precision=4, suppress=True)
print("\nMaking prediction for (mm-normalized) previously unseen inputs of: ")
print(inpts)
print("\nPredicted (mm-normalized) hull resistance is: ")
print(pred)

print("\nEnd Yacht Hull demo")


Making prediction for (mm-normalized) previously unseen inputs of: 
[[0.5611 0.2285 0.9511 0.0011 0.4615 0.9999]]

Predicted (mm-normalized) hull resistance is: 
[[0.9581]]

End Yacht Hull demo
